# Check sentences, looking for templates

## load stuff

In [ ]:
%run ../metrics/report_generation/nlp/cider_idf.py

In [ ]:
%run ../metrics/report_generation/__init__.py

In [ ]:
# run_name = '1108_154558' # lstm-att
run_name = '1113_185718' # SAT
run_id = RunId(run_name, task='rg', debug=False)
run_id

In [ ]:
df_tpl = load_rg_outputs(RunId('1102_190559', False, 'rg'), free=True, labeled=True)
df_tpl.head(1)

In [ ]:
df_tpl = df_tpl[df_tpl['dataset_type'] == 'test']
df_tpl.head(3)

In [ ]:
df = load_rg_outputs(run_id, free=True, best='bleu4', labeled=True)
df.head(2)

In [ ]:
subdf = df.loc[df['dataset_type'] == 'test']
len(subdf)

## Top out reports

In [ ]:
s = Counter(subdf['generated'])

In [ ]:
s = sorted(s.items(), key=lambda x: x[1], reverse=True)
s[:100]

In [ ]:
r = 'frontal and lateral views of the chest were obtained . the lungs are clear without focal consolidation . no pleural effusion or pneumothorax is seen . the cardiac and mediastinal silhouettes are unremarkable .'

In [ ]:
r = 'pa and lateral chest radiographs were obtained . the lungs are well expanded and clear . there is no focal consolidation , effusion , or pneumothorax . the cardiomediastinal silhouette is normal .'

In [ ]:
CHEX_GEN = labels_with_suffix('gen')

In [ ]:
d = subdf.loc[subdf['generated'] == r]
len(d)
d[CHEX_GEN].apply(Counter)

In [ ]:
labels = apply_labeler_to_column([r])
list(zip(CHEXPERT_DISEASES, labels[0]))

In [ ]:
r

## By sentences

In [ ]:
from tqdm import tqdm

In [ ]:
%run ../utils/nlp.py

In [ ]:
_fpath = os.path.join(MIMIC_DIR, 'reports', 'sentences_with_chexpert_labels.csv')
SENTENCES_DF = pd.read_csv(_fpath)
SENTENCES_DF.head(2)

In [ ]:
%%time

d = SENTENCES_DF.set_index('sentence').replace({-2: 0, -1: 1}).transpose().to_dict('list')
d['no acute intrathoracic process .']

In [ ]:
# sentence_counter_abn = Counter()
# sentence_counter_health = Counter()
sentence_counter = Counter()

In [ ]:
actual_diseases = labels_with_suffix('gen')[1:]

In [ ]:
# for index, row in tqdm(df.iterrows(), total=len(df)):
#     report = row['generated']
#     diseases = row[actual_diseases]
#     chosen = sentence_counter_abn if diseases.sum() >= 1 else sentence_counter_health
#     for sentence in split_sentences_text(report):
#         chosen[sentence] += 1

for report in list(df['generated']):
    for sentence in split_sentences_text(report):
        sentence_counter[sentence] += 1
        
len(sentence_counter)

In [ ]:
s = sorted(sentence_counter.items(), key=lambda x: x[1], reverse=True)
s[:10]

In [ ]:
s_h, s_other = [], []
s_by_abn = defaultdict(list)
for tup in s:
    sentence = tup[0]

    labels = d.get(sentence)
    if labels is None:
        s_other.append(tup)
        continue
    
    if np.sum(labels[1:]) == 0:
        s_h.append(tup)
        continue

    for valoration, disease in zip(labels[1:], actual_diseases):
        # TODO: prioritize sentences with more than one valoration == 1??
        if valoration == 1:
            s_by_abn[disease].append(tup)
        
len(s_by_abn), len(s_h), len(s_other)

In [ ]:
s_by_abn['Lung Lesion-gen']

In [ ]:
r = 'again there is enlargement of the cardiac silhouette with some elevation of pulmonary venous pressure and bilateral pleural effusions with compressive atelectasis at the bases .'

In [ ]:
labels = apply_labeler_to_column([r])
list(zip(CHEXPERT_DISEASES, labels[0]))

In [ ]:
l = [
    r
    for r in df['ground_truth']
    if 'in the appropriate clinical setting' in r
]
len(l), l[:2]

## Doc freq

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
matplotlib.rcParams['figure.facecolor'] = 'white'
matplotlib.rcParams['figure.figsize'] = (15, 5)

In [ ]:
%%time

reports = list(df_train['ground_truth'])
doc_freq = compute_doc_freq(reports)
len(doc_freq)

In [ ]:
s = doc_freq

In [ ]:
# s = sorted(s.items(), key=lambda x: x[1], reverse=True)
s[:40]

In [ ]:
get_ngrams = lambda n: [(ngram, amount) for ngram, amount in s if len(ngram) == n]

In [ ]:
unigrams = get_ngrams(1)
bigrams = get_ngrams(2)
trigrams = get_ngrams(3)
quatrigrams = get_ngrams(4)
assert len(unigrams) + len(bigrams) + len(trigrams) + len(quatrigrams) == len(doc_freq)
len(unigrams), len(bigrams), len(trigrams), len(quatrigrams)

In [ ]:
quatrigrams[30:60]
# bigrams[:30]

In [ ]:
amounts = [amount for _, amount in quatrigrams]

In [ ]:
plt.plot(amounts)
plt.yscale('log', base=10)

In [ ]:
l = [(ngram, amount) for ngram, amount in quatrigrams if amount > 0]
len(l)

In [ ]:
l[5000:5050]

In [ ]:
# r = 'non cardiac edema in the lungs .'
# r = 'the hilar and mediastinal contours are within normal limits .'
# r = 'pa and lateral chest compared to xxxx normal heart , lungs , hila , mediastinum and pleural surfaces . no evidence of intrathoracic malignancy or infection .'
r = 'no evidence of intrathoracic malignancy or infection .'

In [ ]:
labels = apply_labeler_to_column([r])
list(zip(CHEXPERT_LABELS, labels[0]))

In [ ]:
from collections import defaultdict

In [ ]:
# target = ('reflects', ',', 'in', 'part')
# target = ('change', 'since', 'prior', 'cxr')
# target = ('silhouette', 'is', 'prominent', 'likely')
# target = ('there', 'is', 'cardiomegaly', '.')
# target = ('effusions', 'are', 'noted', '.')
target = ('mediastinal', 'and', 'hilar', 'contours')
# rr = defaultdict(list)
rr = []
target_n = len(target)
for report in reports:
    words = report.split()
    n_words = len(words)
    for i in range(0, n_words-target_n):
        subreport = set(words[i:i+target_n])
        if all(t in subreport for t in target):
            rr.append(report)
#         count = sum(int(t in subreport) for t in target)
#         if count > 0:
#             rr[count].append(report)
            break
len(rr)

In [ ]:
rr[0]

In [ ]:
rr[0]

## Reverse eng

In [ ]:
%run ../utils/nlp.py

In [ ]:
%run ./analyze_nlp_chexpert/utils.py

In [ ]:
df_tpl = add_nlp_metrics_to_df(df_tpl)
df_tpl.head(2)

In [ ]:
subdf = add_nlp_metrics_to_df(subdf)
subdf.head(2)

In [ ]:
cols = ['ground_truth', 'generated', 'ciderD'] + _GEN_LABELS + _GT_LABELS

In [ ]:
df_tpl = df_tpl[cols].sort_values('ciderD', ascending=False)
df_tpl.head(2)

In [ ]:
subdf = subdf[cols].sort_values('ciderD', ascending=False)
subdf.head(2)

In [ ]:
d = subdf
d = d.loc[((d['No Finding-gen'] == 1) & (d['Support Devices-gen'] == 0))]
d.head(10)

In [ ]:
target = 'Enlarged Cardiomediastinum'
df_tpl.plot(x=f'{target}-gen', y='ciderD', kind='scatter', title=target)

In [ ]:
_GEN_LABELS

In [ ]:
def add_abngroup_col_to_df(df, suffix):
    group_col = []
    
    labels = labels_with_suffix(suffix)[1:] # ignore NF
    to_remove = f'-{suffix}'

    for _, row in df.iterrows():
        abnormalities = row[labels]
        group = ",".join(
            f"{ABN_SHORTCUTS[abn.replace(to_remove, '')]}={value}"
            for abn, value in abnormalities.iteritems()
        )
        group_col.append(group)
        
    return df.assign(**{f'group-{suffix}': group_col})

In [ ]:
df_tpl = add_abngroup_col_to_df(df_tpl, 'gen')
df_tpl = add_abngroup_col_to_df(df_tpl, 'gt')
df_tpl.head(2)

In [ ]:
d = df_tpl
# d = d.loc[d['group-gen'] == d['group-gt']]
score_by_group = d.groupby('group-gen')['ciderD'].apply(sum).sort_values(ascending=True)
score_by_group.head(15)

In [ ]:
target_groups = list(score_by_group.index[:10])
target_groups

In [ ]:
d = df_tpl
d = d.loc[d['group-gen'].isin(target_groups)]
e = d.loc[d['group-gen'] == d['group-gt']] # Correct ones
f'{len(e)}/{len(d)}', target_group, d['ciderD'].values

In [ ]:
d = df_tpl
d = d.sort_values('ciderD', ascending=True)
d = d.loc[d['group-gen'] == d['group-gt']]

In [ ]:
c = defaultdict(list)
for _, row in d.iterrows():
    report = row['ground_truth']
    ciderD = row['ciderD']
    for sentence in split_sentences_text(report):
        c[sentence].append(ciderD)
len(c)

In [ ]:
cc = sorted(c.items(), key=lambda x: sum(x[1]), reverse=False)
cc[:10]

In [ ]:
cc[250:260]

In [ ]:
# r = 'mediastinal and hilar silhouettes are unremarkable and no pleural abnormality .'
# r = 'no pleural abnormality .'
# r = 'there is no pneumothorax , vascular congestion , or pleural effusion .'
# r = 'there is no evidence of pleural effusions .'
# r = 'the lungs are clear without infiltrate or effusion .'
# r = 'there is no pulmonary edema , consolidation , or pleural effusion .'
# r = 'in comparison with the study of xxxx , there is again some enlargement of the cardiac silhouette without definite vascular congestion , pleural effusion , or acute focal pneumonia .'
# r = 'the cardiomediastinal silhouette , pulmonary vasculature , and aorta are within normal limits .'
# r = 'cardiomediastinal and hilar silhouettes and pleural surfaces are normal .'
# r = 'the heart is normal in size and there is no vascular congestion , pleural effusion , or acute focal pneumonia .'
r = 'the heart is normal in size , and the mediastinal contours are normal .'
l = apply_labeler_to_column([r])
list(zip(CHEXPERT_DISEASES, l[0]))

In [ ]:
c = Counter()
for report in d['ground_truth']:
    for sentence in split_sentences_text(report):
        c[sentence] += 1
        pass
c

In [ ]:
cc = sorted(c.items(), key=lambda x: x[1], reverse=True)
cc[:10]

## Check sentences by most common groups

In [ ]:
%run ../utils/nlp.py

In [ ]:
_fpath = os.path.join(MIMIC_DIR, 'reports', 'sentences_with_chexpert_labels.csv')
SENTENCES_DF = pd.read_csv(_fpath)
SENTENCES_DF.head(2)

In [ ]:
d = df.loc[df['dataset_type'] == 'test']
d.head(2)

In [ ]:
reports = list(d['ground_truth'])
sentences = [s for r in reports for s in split_sentences_text(r)]
len(reports), len(sentences)

In [ ]:
labeler = ChexpertLabeler(fill_empty=-2, fill_uncertain=-1, caller_id='notebook')
labeler = CacheLookupLabeler(labeler, SENTENCES_DF, text_key='sentence')
labeler = NBatchesLabeler(labeler)
labeler = AvoidDuplicatedLabeler(labeler)
labeler

In [ ]:
%%time
labels = labeler(sentences)
labels.shape

In [ ]:
sentences_df = pd.DataFrame(labels, index=sentences, columns=CHEXPERT_DISEASES)
sentences_df = sentences_df.reset_index().rename(columns={'index': 'sentence'})
sentences_df.head(2)

In [ ]:
# Save it for convenience
_fpath = os.path.join(MIMIC_DIR, 'reports', 'sentences_with_chexpert_labels.new_test.csv')
assert not os.path.isfile(_fpath)
# sentences_df.to_csv(_fpath, index=False)

In [ ]:
sentences_df = pd.read_csv(_fpath).replace({-1: 1})
sentences_df.head(2)

In [ ]:
def add_abngroup_col_to_df(df, targets=(0,1)):
    group_col = []
    
    labels = CHEXPERT_DISEASES[1:] # ignore NF

    for _, row in df.iterrows():
        abnormalities = row[labels]
        group = ";".join(
            f"{ABN_SHORTCUTS[abn]}={value}"
            for abn, value in abnormalities.iteritems()
            if value in targets
        )
        group_col.append(group)
        
    return df.assign(**{'group': group_col})

In [ ]:
sentences_df = add_abngroup_col_to_df(sentences_df)
sentences_df.head(2)

In [ ]:
cols = ['sentence', 'group']
df = sentences_df[cols]
df.head(2)

In [ ]:
e = df.groupby('sentence')['group'].apply(lambda x: len(set(x.values)))
(e == 1).all()

In [ ]:
# df['appearances'] = df.groupby('sentence')['sentence'].transform('size')
df['appearances'] = df['sentence'].map(df['sentence'].value_counts())
df.head(2)

In [ ]:
df = df.groupby('sentence').first().reset_index().sort_values('appearances', ascending=False)
df.head(2)

In [ ]:
n_pos, n_neg = zip(*[
    (group.count('1'), group.count('0'))
    for group in df['group']
])
df = df.assign(n_pos=n_pos, n_neg=n_neg)
df.head(2)

In [ ]:
df.head(10)

In [ ]:
d = df
ready = set(['Pt=0;PE=0','Cons=0;Pt=0;PE=0','EC=1;Card=1','Cons=0;Pt=0','EC=0;Card=0'])
# d = d.loc[~d['group'].isin(ready)]
d = d.loc[d['appearances'] >= 2]
# d = d.loc[d['n_pos'] >= 2]
d = d.loc[d['group'].str.contains(r'\bE=1')]
# d = d.loc[(d['n_neg'] + d['n_pos'] >= 2)]
d = d.sort_values(['n_neg', 'appearances'], ascending=False)
print(len(d))
d.head(20)

In [ ]:
list(d['sentence'])

In [ ]:
df_tpl['No Finding-gen'].value_counts()

In [ ]:
dd = df_tpl
dd = dd.loc[dd['No Finding-gen'] == 0]
print(len(dd))
dd.head(10)

In [ ]:
dd['No Finding-gt'].value_counts()

In [ ]:
dd2 = dd
# dd2 = dd2.loc[dd2['No Finding-gt'] == 0]
# Correct ones:
dd2 = dd2.loc[dd2[_GEN_LABELS].eq(dd2[_GT_LABELS].rename(columns=dict(zip(_GT_LABELS, _GEN_LABELS))), axis='index').all(axis=1)]
dd2 = dd2.loc[dd2['ciderD'] <= 0.4]
print(len(dd2))
# dd2.plot(x='No Finding-gt', y='ciderD', kind='hist', bins=20)

In [ ]:
dd2 = dd2.sort_values('ground_truth', key=lambda x: x.str.len(), ascending=True)
dd2.head(20)

In [ ]:
ss = Counter([s for r in dd2['ground_truth'] for s in split_sentences_text(r)])
ss = sorted(ss.items(), key=lambda x: x[1], reverse=True)
ss[:10]

In [ ]:
list(dd2['ground_truth'])

## Sanity check train-val

In [ ]:
df_tpl = load_rg_outputs(RunId('1102_190559', False, 'rg'), free=True, labeled=True)
df_tpl.head(1)

In [ ]:
%run ../models/report_generation/templates/chex_group_mimic.py

In [ ]:
dtrain = df_tpl.loc[df_tpl['dataset_type'] == 'train']
len(dtrain)

In [ ]:
sent_counter = Counter([s for r in dtrain['ground_truth'] for s in split_sentences_text(r)])
# sent_counter = dict(sorted(sent_counter.items(), key=lambda x: x[1], reverse=True))
sent_counter['m .']

In [ ]:
appears = lambda text: sum(c for r, c in sent_counter.items() if text in r)

In [ ]:
for group in GROUPS_v8:
    sentence = group[2]
    print(sentence, appears(sentence))

In [ ]:
for abn, d in CHEX_mimic_single.items():
    print(abn)
    for v, sent in d.items():
        if sent:
            print(f'\t"{sent}": {appears(sent)}')

In [ ]:
_fpath = os.path.join(MIMIC_DIR, 'reports', 'sentences_with_chexpert_labels.csv')
SENTENCES_DF = pd.read_csv(_fpath).replace({-1:1})
SENTENCES_DF['appearances'] = [sent_counter.get(sent, 0) for sent in SENTENCES_DF['sentence']]
SENTENCES_DF.head(2)

In [ ]:
d = SENTENCES_DF
target = 'Lung Lesion'
others = list(CHEXPERT_DISEASES)
others.remove(target)
d = d.loc[((d[target] == 1) & (d[others] == -2).all(axis=1))]
print(len(d))
d = d.sort_values(['appearances', 'sentence'], ascending=False)
d = d[['sentence', 'appearances']]
d.head(10)

In [ ]:
list(d['sentence'])

In [ ]:
[(r['sentence'], r['appearances']) for _, r in d.iterrows() if 'fibrosis' in r['sentence']]

In [ ]:
# SENTENCES_DF = add_abngroup_col_to_df(SENTENCES_DF)
SENTENCES_DF.head(2)

## Re-eval with chex

In [ ]:
%run ../models/report_generation/templates/chex_group_mimic.py

In [ ]:
sentences = [s for d in CHEX_mimic_single.values() for s in d.values() if s]
sentences

In [ ]:
labels = apply_labeler_to_column(sentences)
labels

In [ ]:
sentences = [group[2] for group in GROUPS_v8]
sentences

In [ ]:
labels = apply_labeler_to_column(sentences)
labels

In [ ]:
# df_single = load_rg_outputs(RunId('1102_190559', False, 'rg'), free=True, labeled=True)
df_single = df_single.loc[df_single['dataset_type'] == 'test']
df_single.head(1)

In [ ]:
# df_grouped = load_rg_outputs(RunId('1129_191853', False, 'rg'), free=True, labeled=True)
df_grouped = df_grouped.loc[df_grouped['dataset_type'] == 'test']
df_grouped.head(1)

In [ ]:
cols = ['image_fname', 'ground_truth', 'generated', 'No Finding-gt', 'No Finding-gen']
df1 = df_single[cols].reset_index(drop=True)
df2 = df_grouped[cols].reset_index(drop=True)
assert len(df1) == len(df2)
len(df1), len(df1['image_fname'].unique())

In [ ]:
df1.head(2)

In [ ]:
df2.head(2)

In [ ]:
merged = df1.merge(df2, on=['image_fname', 'ground_truth', 'No Finding-gt'],
                   how='inner', validate='one_to_one')
assert len(merged) == len(df1), f'{len(merged)} vs {len(df1)}'

In [ ]:
mismatch = merged.loc[merged['No Finding-gen_x'] != merged['No Finding-gen_y']]
mismatch.head(2)

In [ ]:
mismatch['No Finding-gt'].value_counts()

In [ ]:
len(mismatch)

In [ ]:
single = list(set(mismatch['generated_x']))
grouped = list(set(mismatch['generated_y']))
len(single), len(grouped)

In [ ]:
single = single[0]
grouped = grouped[0]
single, grouped

In [ ]:
r = 'there is no focal consolidation , pleural effusion , or pneumothorax . heart size is normal . the cardiomediastinal silhouette is normal . there is no evidence of fibrosis . no displaced fracture is seen . tube in standard placement .'    
labels = apply_labeler_to_column([single, grouped, r], fill_empty=-3)
labels

In [ ]:
list(zip(CHEXPERT_DISEASES, zip(*labels)))

In [ ]:
len([r for r in df_single['ground_truth'] if 'tracheostomy' in r])